In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time

%matplotlib inline
pd.set_option('max_colwidth',1000)

In [2]:
# Load the loan data
df = pd.read_pickle('data_cleaned.pkl')
df.reset_index(drop=True, inplace=True)

# Remove loan_status as it is one of the items we will predict
df.drop('loan_status', axis=1, inplace=True)

In [3]:
# Let's look at the type of all the columns
for y in df.columns:
    print(y, df[y].dtype)

id float64
member_id float64
loan_amnt int64
funded_amnt int64
funded_amnt_inv float64
term int64
int_rate float64
installment float64
grade object
sub_grade object
emp_length object
home_ownership object
annual_inc float64
verification_status object
pymnt_plan object
purpose object
zip_code int64
addr_state object
dti float64
delinq_2yrs int64
inq_last_6mths int64
mths_since_last_delinq int64
mths_since_last_record int64
open_acc int64
pub_rec int64
revol_bal int64
revol_util float64
total_acc int64
initial_list_status object
out_prncp float64
out_prncp_inv float64
total_pymnt float64
total_pymnt_inv float64
total_rec_prncp float64
total_rec_int float64
total_rec_late_fee float64
recoveries float64
collection_recovery_fee float64
last_pymnt_amnt float64
collections_12_mths_ex_med int64
mths_since_last_major_derog int64
application_type object
acc_now_delinq int64
tot_coll_amt int64
tot_cur_bal int64
open_acc_6m int64
open_act_il int64
open_il_12m int64
open_il_24m int64
mths_since_rcn

In [ ]:
# Categorical features from data-cleanup.ipynb
cat_features = ['grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 
                'purpose', 'addr_state', 'initial_list_status', 'application_type']

for y in cat_features:
    print(y)
    df = df.join(pd.get_dummies(df[y], prefix=y))
    df.drop(y, axis=1, inplace=True)

In [ ]:
# Split the data in features and target label
int_rate = df['int_rate']
features = df.drop('int_rate', axis=1)

In [ ]:
from sklearn.cross_validation import train_test_split

# Split the features and int_rate data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, int_rate, test_size = 0.2, random_state = 0)

print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

In [ ]:
from sklearn.metrics import mean_squared_error
import math

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    start = time() # Get start time
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
       
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples
    results['rmsd_train'] = math.sqrt(mean_squared_error(y_train, learner.predict(X_train)))
        
    # TODO: Compute accuracy on test set
    results['rmsd_test'] = math.sqrt(mean_squared_error(y_test, learner.predict(X_test)))
    
    # TODO: Compute accuracy on test set
    results['score_train'] = learner.score(X_train, y_train)
    results['score_test'] = learner.score(X_test, y_test)
    
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
    
    y_test.reset_index(drop=True, inplace=True)
    join_df = pd.concat([y_test, pd.Series(learner.predict(X_test))], axis=1)
    print(join_df)
        
    # Return the results
    return results

In [ ]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

result = train_predict(clf, len(y_train), X_train, y_train, X_test, y_test)
result